# Six Degrees of Kevin Bacon

![](images/Kevin_Bacon.jpg)

This activity is motivated by the text **Web Scraping with Python** by Ryan Mitchell, available through O'Reilly [here](http://shop.oreilly.com/product/0636920078067.do).  This book goes in depth with much more on using different libraries with Python around common webscraping tasks and I highly recommend it.  We will focus on the activity of moving from a base page to further pages through their links.  

In [2]:
import requests
from bs4 import BeautifulSoup

### Scraping Links

Below, we take the page dealing with the six degrees of Keving Bacon problem.  Here, our goal is to extract links to other pages that we will subsequently pass to requests.  Recall that a link is located in an `<a>` tag and the link is contained in the `href` attribute.  For example, the tag

```HTML
<a href="/wiki/Six_degrees_of_separation" title="Six degrees of separation">six degrees of separation</a>
```

references the Six Degrees of Separation article.  Note that this is a url within Wikipedia.  We can isolate these inner Wikipedia references.  To begin, let's inspect the link content.

In [3]:
response = requests.get('https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon')
#what we get back when we navigate to this page in our browser

In [4]:
soup = BeautifulSoup(response.text, 'html.parser')
#turnin it into SOUP !

In [13]:
soup.find('a')
#returns the first A tag on th webiste
#returns a list - everything in the list is a tag itself and it has attributes

<a id="top"></a>

In [18]:
soup.find_all('a')[:10]

[<a id="top"></a>,
 <a class="mw-jump-link" href="#mw-head">Jump to navigation</a>,
 <a class="mw-jump-link" href="#p-search">Jump to search</a>,
 <a class="image" href="/wiki/File:Kevin_Bacon.jpg"><img alt="" class="thumbimage" data-file-height="461" data-file-width="369" height="275" src="//upload.wikimedia.org/wikipedia/commons/thumb/d/d2/Kevin_Bacon.jpg/220px-Kevin_Bacon.jpg" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/d/d2/Kevin_Bacon.jpg/330px-Kevin_Bacon.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/d/d2/Kevin_Bacon.jpg 2x" width="220"/></a>,
 <a class="internal" href="/wiki/File:Kevin_Bacon.jpg" title="Enlarge"></a>,
 <a class="mw-redirect" href="/wiki/Parlor_game" title="Parlor game">parlor game</a>,
 <a href="/wiki/Six_degrees_of_separation" title="Six degrees of separation">six degrees of separation</a>,
 <a href="/wiki/Kevin_Bacon" title="Kevin Bacon">Kevin Bacon</a>,
 <a href="/wiki/Hollywood" title="Hollywood">Hollywood</a>,
 <a href="/wiki/Kevin_Bacon" ti

In [19]:
for link in soup.find_all('a')[:10]:
    print (link.attrs)
#printing different attributes for these links
#href, class, and title?
#we are most interested in the hrefs -> which take us to another page related to wikipedia

{'id': 'top'}
{'class': ['mw-jump-link'], 'href': '#mw-head'}
{'class': ['mw-jump-link'], 'href': '#p-search'}
{'href': '/wiki/File:Kevin_Bacon.jpg', 'class': ['image']}
{'href': '/wiki/File:Kevin_Bacon.jpg', 'class': ['internal'], 'title': 'Enlarge'}
{'href': '/wiki/Parlor_game', 'class': ['mw-redirect'], 'title': 'Parlor game'}
{'href': '/wiki/Six_degrees_of_separation', 'title': 'Six degrees of separation'}
{'href': '/wiki/Kevin_Bacon', 'title': 'Kevin Bacon'}
{'href': '/wiki/Hollywood', 'title': 'Hollywood'}
{'href': '/wiki/Kevin_Bacon', 'title': 'Kevin Bacon'}


In [5]:
for link in soup.find_all('a')[:10]:
    if 'href' in link.attrs: #if href is in links attr 
        print(link.attrs['href']) #only print what that link is

#mw-head
#p-search
/wiki/File:Kevin_Bacon.jpg
/wiki/File:Kevin_Bacon.jpg
/wiki/Parlor_game
/wiki/Six_degrees_of_separation
/wiki/Kevin_Bacon
/wiki/Hollywood
/wiki/Kevin_Bacon


Okay, seems there are links outside of the inner wiki links.  However, we see that the wiki links contain `/wiki/`, no colons, and the links are all within the body of the page.  Exploiting these means we can write a regular expression 

```
^(/wiki/)((?!:).)*$
```

that will match only the wiki links.  

In [6]:
import re

In [7]:
for link in soup.find('div', {'id': 'bodyContent'}).find_all('a', href = re.compile('^(/wiki/)((?!:).)*$'))[:10]:
    if 'href' in link.attrs:
        print(link.attrs['href'])
        
        #div tag is what we want - only grabbing them that have the id: bodycontent
        #within that we are going to only grab the ones that are what we want
        #^(/wiki/)((?!:).)*$ -->
            #arrow means contains --> [^X] would mean doesn't have an X in it
            #

/wiki/Parlor_game
/wiki/Six_degrees_of_separation
/wiki/Kevin_Bacon
/wiki/Hollywood
/wiki/Kevin_Bacon
/wiki/Kevin_Bacon
/wiki/Charitable_organization
/wiki/SixDegrees.org
/wiki/Premiere_(magazine)
/wiki/Kevin_Bacon


### A Function for Links

Now, let's write a function that extracts the link from any wikipedia page.  We should be able to use the idea that the links we care about are located in the same place as our Six Degrees example.  

In [8]:
def get_wikilinks(url):
    links = []
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    for link in soup.find('div', {'id': 'bodyContent'}).find_all('a', href = re.compile('^(/wiki/)((?!:).)*$')):
        links.append(link)
    return links

In [9]:
links = get_wikilinks('https://en.wikipedia.org/wiki/Kevin_Bacon')

In [10]:
links[:10]

[<a class="mw-disambig" href="/wiki/Kevin_Bacon_(disambiguation)" title="Kevin Bacon (disambiguation)">Kevin Bacon (disambiguation)</a>,
 <a href="/wiki/San_Diego_Comic-Con" title="San Diego Comic-Con">San Diego Comic-Con</a>,
 <a href="/wiki/Philadelphia" title="Philadelphia">Philadelphia</a>,
 <a href="/wiki/Pennsylvania" title="Pennsylvania">Pennsylvania</a>,
 <a href="/wiki/Kyra_Sedgwick" title="Kyra Sedgwick">Kyra Sedgwick</a>,
 <a href="/wiki/Sosie_Bacon" title="Sosie Bacon">Sosie Bacon</a>,
 <a href="/wiki/Edmund_Bacon_(architect)" title="Edmund Bacon (architect)">Edmund Bacon</a>,
 <a href="/wiki/Michael_Bacon_(musician)" title="Michael Bacon (musician)">Michael Bacon</a>,
 <a href="/wiki/Footloose_(1984_film)" title="Footloose (1984 film)">Footloose</a>,
 <a href="/wiki/JFK_(film)" title="JFK (film)">JFK</a>]

### Connecting Pages

Now, we want to follow these references, gather more urls, and repeat. For the sake of not running to exhaustion, I abbreviate the output using only a large length requirement for the link list.  To traverse all the pages we would simply change the 

```python
while len(links) > 100:
```

to 

```python
while len(links) > 0:
```

In [11]:
import random
def get_wikilinks(url):
    links = []
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    for link in soup.find('div', {'id': 'bodyContent'}).find_all('a', href = re.compile('^(/wiki/)((?!:).)*$')):
        links.append(link)
    return links

In [21]:
links = get_wikilinks('https://en.wikipedia.org/wiki/Kevin_Bacon')
while len(links) > 100:
    newArticle = 'https://en.wikipedia.org' + links[random.randint(0, len(links)-1)].attrs['href']
    print(newArticle)
    links = get_wikilinks(newArticle)
    #create a new link i guess?

https://en.wikipedia.org/wiki/Kevin_Costner
https://en.wikipedia.org/wiki/Bull_Durham
https://en.wikipedia.org/wiki/Bull_Durham_(disambiguation)


### Problem

Write a function to retrieve a list of albums of any area you are interested in using Wikipedia's list of list of albums page: https://en.wikipedia.org/wiki/Lists_of_albums.

In [27]:
response = requests.get('https://en.wikipedia.org/wiki/List_of_grunge_albums')

In [28]:
soup = BeautifulSoup(response.text, 'html.parser')

In [32]:
soup.find('h2') #first header bang bam boom

<h2>Navigation menu</h2>

In [34]:
song_table = soup.find_all('table', {'class':'wikitable'}) #chose this by inspecting the object and seeing what the makeup of the object is - <table class=wikitable>

In [35]:
song_table

[<table class="wikitable">
 <tbody><tr>
 <th>Artist/Band</th>
 <th>Title</th>
 <th>Label</th>
 <th>Notes
 </th></tr>
 <tr>
 <td colspan="7" scope="row" style="text-align:left; background:#e9e9e9"><b><a href="/wiki/1984_in_music" title="1984 in music">1984</a></b>
 </td></tr>
 <tr>
 <td><a href="/wiki/The_U-Men" title="The U-Men">The U-Men</a></td>
 <td><i><a href="/wiki/U-Men_(album)" title="U-Men (album)">U-Men</a></i></td>
 <td><a class="new" href="/w/index.php?title=Bomb_Shelter_Records&amp;action=edit&amp;redlink=1" title="Bomb Shelter Records (page does not exist)">Bomb Shelter</a></td>
 <td>Debut release
 </td></tr>
 <tr>
 <td colspan="7" scope="row" style="text-align:left; background:#e9e9e9"><b><a href="/wiki/1985_in_music" title="1985 in music">1985</a></b>
 </td></tr>
 <tr>
 <td><a href="/wiki/The_U-Men" title="The U-Men">The U-Men</a></td>
 <td><i><a href="/wiki/Stop_Spinning" title="Stop Spinning">Stop Spinning</a></i></td>
 <td><a href="/wiki/Homestead_Records" title="Home

In [37]:
for result in song_table:
    print (result.find('tr').text) #tr is table row --> could find all get all of them: find_all('tr')
    #.text only returns the title within the tr tags


Artist/Band
Title
Label
Notes



In [39]:
songs = []
for result in song_table:
    print(result.find('td').text)

1984

